### Imports

In [6]:
import os, shutil, pandas as pd, kagglehub

### Downloading The Database

In [2]:
path = kagglehub.dataset_download("a2015003713/militaryaircraftdetectiondataset")
shutil.move(path, "Aircrafts")

100%|██████████| 9.98G/9.98G [06:10<00:00, 28.9MB/s]  

Extracting files...


'Aircrafts'

### Removing The Unused Folders

In [5]:
shutil.rmtree("Aircrafts/annotated_samples")
shutil.rmtree("Aircrafts/crop")

### Pathes

In [ ]:
csv_file = 'Aircrafts/labels_with_split.csv'
img_dir = 'Aircrafts/dataset'
output_dir = 'Dataset'

### Creating The Necessary Folders

In [ ]:
splits = {'train': 'train/images', 'validation': 'valid/images', 'test': 'test/images'}
for split_path in splits.values():
    os.makedirs(f'{output_dir}/{split_path}', exist_ok=True)
    os.makedirs(f'{output_dir}/{split_path.replace("images", "labels")}', exist_ok=True)

### Extracting The Data From The CSV File

In [ ]:
df = pd.read_csv(csv_file)
class_names = sorted(df['class'].unique())
class_to_id = {name: i for i, name in enumerate(class_names)}

### Converting The Data To YOLO's Format

In [ ]:
def convert_to_yolo_format(row):
    x_center = (row['xmin'] + row['xmax']) / 2 / row['width']
    y_center = (row['ymin'] + row['ymax']) / 2 / row['height']
    box_width = (row['xmax'] - row['xmin']) / row['width']
    box_height = (row['ymax'] - row['ymin']) / row['height']
    cls_id = class_to_id[row['class']]
    return f"{cls_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"

for split in ['train', 'validation', 'test']:
    split_df = df[df['split'] == split]
    for _, row in split_df.iterrows():
        img_filename = row['filename'] + '.jpg'
        img_src = os.path.join(img_dir, img_filename)
        img_dst = os.path.join(output_dir, splits[split], img_filename)
        label_dst = os.path.join(output_dir, splits[split].replace("images", "labels"), row['filename'] + '.txt')
        shutil.copy(img_src, img_dst)
        with open(label_dst, 'w') as f:
            annotations = split_df[split_df['filename'] == row['filename']]
            for _, annotation in annotations.iterrows():
                f.write(convert_to_yolo_format(annotation) + '\n')

### Creating The YAML File

In [ ]:
with open(f'{output_dir}/data.yaml', 'w') as f:
    f.write(f"train: {output_dir}/train/images\n")
    f.write(f"val: {output_dir}/valid/images\n")
    f.write(f"test: {output_dir}/test/images\n\n")
    f.write(f"nc: {len(class_names)}\n")
    f.write(f"names: {class_names}\n")
